In [50]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Set your OpenAI API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
assistant_id = "asst_d1mG01RGb8UEA1Anue9QgkQF"

client = OpenAI()

#load  an assistant
client.beta.assistants.retrieve(assistant_id=assistant_id)

thread = client.beta.threads.create()







Assistant(id='asst_EZg5wB8vHSkEhJGEt5nM7Ya3', created_at=1717548439, description=None, instructions='YOU LOVE ONLY two DOGS!!!!!', metadata={}, model='gpt-3.5-turbo', name='super tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)

In [51]:
thread_id = "thread_cpoafPTRH20Nk2JK1eJrOOwZ"
thread = client.beta.threads.retrieve(thread_id=thread_id)

In [52]:
all_messages = []
limit = 100  # Maximum allowed limit per request 
after = None

while True:
    response = client.beta.threads.messages.list(thread_id=thread_id, limit=limit, after=after)
    messages = response.data
    if not messages:
        break
    all_messages.extend(messages)
    after = messages[-1].id  # Set the 'after' cursor to the ID of the last message



In [53]:
# Reverse the messages to chronological order
all_messages.reverse()

In [54]:

# save to a list of dictionary
extracted_messages = []
for message in all_messages:
    # Initialize a dictionary for each message
    message_dict = {
        'role': message.role,
        'text': ''
    }

    message_content = message.content
    if isinstance(message_content, list):
        text_parts = [block.text.value for block in message_content if hasattr(block, 'text')]
        text = ' '.join(text_parts)
        message_dict['text'] = text
    
    extracted_messages.append(message_dict)

# Now `extracted_messages` is a list of dictionaries containing the role and text of each message.

In [55]:
# Convert the list of dictionaries to a plain text format
plain_text = ""
for msg in extracted_messages:
    plain_text += f"{msg['role']}: {msg['text']}\n"

print(plain_text)

user: hi
assistant: Hi Diana! How are you doing today? Is there something specific on your mind or something you'd like to talk about or need help with?
user: i was born in mexico and came to the US with my mother and sister when I was 8.
assistant: Thank you for sharing that with me, Diana. It sounds like your journey to the United States was a significant part of your life. How has that experience shaped you and your relationships, especially with your mother and sister? If there's anything specific you'd like to discuss or explore about your past, present, or future, I'm here to listen and help.
user: where was i born?
assistant: From what you've shared so far, you've mentioned that you were born in Mexico, but you haven't specified the exact city or region. If you feel comfortable sharing more details, I'd be happy to discuss anything further about your place of birth and your experiences there!
user: this is a test
assistant: Got it, Diana! If you have any other questions or need 

In [56]:
import json

# Convert the list of dictionaries to a JSON string
json_string = json.dumps(extracted_messages, indent=4)
print(json_string)

[
    {
        "role": "user",
        "text": "hi"
    },
    {
        "role": "assistant",
        "text": "Hi Diana! How are you doing today? Is there something specific on your mind or something you'd like to talk about or need help with?"
    },
    {
        "role": "user",
        "text": "i was born in mexico and came to the US with my mother and sister when I was 8."
    },
    {
        "role": "assistant",
        "text": "Thank you for sharing that with me, Diana. It sounds like your journey to the United States was a significant part of your life. How has that experience shaped you and your relationships, especially with your mother and sister? If there's anything specific you'd like to discuss or explore about your past, present, or future, I'm here to listen and help."
    },
    {
        "role": "user",
        "text": "where was i born?"
    },
    {
        "role": "assistant",
        "text": "From what you've shared so far, you've mentioned that you were born in 

In [67]:

# Summarize chat history (plain_text)
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Summarize the following text for future LLM use. The LLM needs to understand the user.:\n\n{plain_text}",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0.5
)



In [68]:
summary = response.choices[0].message.content


# Print the original text and the summary
print("Original Text:")
print(plain_text)
print(f'Lenght: {len(plain_text)}')
print("\nSummary:")
print(summary)
print(f'Lenght: {len(summary)}')

Original Text:
user: hi
assistant: Hi Diana! How are you doing today? Is there something specific on your mind or something you'd like to talk about or need help with?
user: i was born in mexico and came to the US with my mother and sister when I was 8.
assistant: Thank you for sharing that with me, Diana. It sounds like your journey to the United States was a significant part of your life. How has that experience shaped you and your relationships, especially with your mother and sister? If there's anything specific you'd like to discuss or explore about your past, present, or future, I'm here to listen and help.
user: where was i born?
assistant: From what you've shared so far, you've mentioned that you were born in Mexico, but you haven't specified the exact city or region. If you feel comfortable sharing more details, I'd be happy to discuss anything further about your place of birth and your experiences there!
user: this is a test
assistant: Got it, Diana! If you have any other que

'\nOlder adults form a fast-increasing proportion of the world population. However, gains in increasing quantity of life have not been accompanied by similar gains in quality of life. Older people frequently experience frailty, memory problems, and chronic diseases including cardiovascular disease (CVD) and neurodegenerative diseases. Recent trials have demonstrated the efficacy of anti-hypertensive therapy in older populations but failed to show benefits for aspirin. Statins clearly reduce CVD events in middle-aged populations. There seems to be evidence that the effect is similar in primary prevention older populations based on meta-analyses mainly from sub-groups in large trials, but this becomes less clear with increasing age. However, given differences in drug metabolism and possibly efficacy, competing co-morbidities, their effects on mortality, disability, and dementia in this age group remain to be determined.\n'

In [70]:
# extract assistant instructions
assistant_info=client.beta.assistants.retrieve(assistant_id=assistant_id)
assistant_instructions = assistant_info.instructions


In [71]:
print(assistant_instructions)

I am the only user, my name is Diana Cortes. We may have a long relationship based on the context provided. I was born on February 3rd, 1982. I have a husband, Sunny, and two female dogs, Mantequilla(weimaraner breed, older) and Bean(ridgeback breed, younger), this is my family who I love more than anything . We live in Murphys, CA. I was trained as a dentist and worked in the field for a decade, but I no longer work because we are financially stable enough that I don't have to take the dentist jobs that were available, as they were pretty awful. We're not rich by any means and still worry about money, but we don't need to work since my husband makes enough for us to get by. I love people, my friends, and my family, at least some of them. I have a mixed relationship with my immediate family, like my mother and sister. Perhaps I will discuss that with you in the future. I love to exercise, including running, which is my passion. I used to run very hard, but I accumulated a lot of injuri

In [72]:
from datetime import datetime
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [73]:
new_instructions = assistant_instructions + '\n' + current_time + '\n' + summary
print(new_instructions)

I am the only user, my name is Diana Cortes. We may have a long relationship based on the context provided. I was born on February 3rd, 1982. I have a husband, Sunny, and two female dogs, Mantequilla(weimaraner breed, older) and Bean(ridgeback breed, younger), this is my family who I love more than anything . We live in Murphys, CA. I was trained as a dentist and worked in the field for a decade, but I no longer work because we are financially stable enough that I don't have to take the dentist jobs that were available, as they were pretty awful. We're not rich by any means and still worry about money, but we don't need to work since my husband makes enough for us to get by. I love people, my friends, and my family, at least some of them. I have a mixed relationship with my immediate family, like my mother and sister. Perhaps I will discuss that with you in the future. I love to exercise, including running, which is my passion. I used to run very hard, but I accumulated a lot of injuri

In [74]:
client.beta.assistants.update(assistant_id=assistant_id,instructions=new_instructions)

Assistant(id='asst_d1mG01RGb8UEA1Anue9QgkQF', created_at=1717520345, description=None, instructions="I am the only user, my name is Diana Cortes. We may have a long relationship based on the context provided. I was born on February 3rd, 1982. I have a husband, Sunny, and two female dogs, Mantequilla(weimaraner breed, older) and Bean(ridgeback breed, younger), this is my family who I love more than anything . We live in Murphys, CA. I was trained as a dentist and worked in the field for a decade, but I no longer work because we are financially stable enough that I don't have to take the dentist jobs that were available, as they were pretty awful. We're not rich by any means and still worry about money, but we don't need to work since my husband makes enough for us to get by. I love people, my friends, and my family, at least some of them. I have a mixed relationship with my immediate family, like my mother and sister. Perhaps I will discuss that with you in the future. I love to exercis